In [25]:
import pandas as pd

In [35]:
def obj_to_num(df,cols):
    def obj_to_num(row):
        row = row.replace(',','')
        return float(row)

    for col in cols: 
        df[col] = df[col].apply(obj_to_num) 
    return df
    
def prepare_data(file):
  dateparse = lambda x: pd.datetime.strptime(x, '%b %d, %Y').strftime('%Y-%m-%d')
  data  = pd.read_csv(file, parse_dates=[0], date_parser=dateparse)
  columns = ['Open','High','Low','Close','Volume','Market Cap']
  num_data = obj_to_num(data,columns)
  data['Date'] = data['Date'].dt.tz_localize(None)
  data = data.groupby([pd.Grouper(key='Date', freq='H')]).first().reset_index()
  data = data.set_index('Date')
  data = data[['High']]
  data.dropna(subset=['High'], inplace=True)
  return data

In [41]:
file = 'full_data_eth_20072020.csv'
data = prepare_data(file)
data.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: FutureWarning:

The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime instead.



,High
Date,
2015-08-07,3.540000
2015-08-08,2.800000
2015-08-09,0.879810
2015-08-10,0.729854
2015-08-11,1.130000


In [28]:
data.index

DatetimeIndex(['2015-08-07', '2015-08-08', '2015-08-09', '2015-08-10',
               '2015-08-11', '2015-08-12', '2015-08-13', '2015-08-14',
               '2015-08-15', '2015-08-16',
               ...
               '2020-07-11', '2020-07-12', '2020-07-13', '2020-07-14',
               '2020-07-15', '2020-07-16', '2020-07-17', '2020-07-18',
               '2020-07-19', '2020-07-20'],
              dtype='datetime64[ns]', name='Date', length=1810, freq=None)

In [9]:
!pip install chart_studio

     |████████████████████████████████| 71kB 2.0MB/s 


In [15]:
import plotly.graph_objects as go
import chart_studio as py

import seaborn as sns

from statsmodels.tsa.seasonal import seasonal_decompose


In [29]:
py.tools.set_credentials_file(username='xxx...', api_key='xxx...')

In [43]:
trace1 = go.Scatter(
    x = data_plot1.index,
    y = data_plot1['High'].astype(float),
    mode = 'lines',
    name = 'ETH Price (High)'
)

layout = dict(
    title='Historical ETH Close price (USD) (...-2020)',
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1,
                     label='1m',
                     step='month',
                     stepmode='backward'),
                dict(count=6,
                     label='6m',
                     step='month',
                     stepmode='backward'),
                dict(count=12,
                     label='1y',
                     step='month',
                     stepmode='backward'),
                dict(count=36,
                     label='3y',
                     step='month',
                     stepmode='backward'),
                dict(step='all')
            ])
        ),
        rangeslider=dict(
            visible = True
        ),
        type='date'
    )
)

data = [trace1]
fig = dict(data = data.copy(), layout=layout)
py.plotly.iplot(fig, filename = "Time Series with Rangeslider")

In [46]:
!pip install pyramid-arima
from pyramid.arima import auto_arima

In [47]:
file = 'full_data_eth_20072020.csv'
data = prepare_data(file)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: FutureWarning:

The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime instead.



In [48]:
stepwise_model = auto_arima(data, start_p=1, start_q=1,
                           max_p=3, max_q=3, m=12,
                           start_P=0, seasonal=True,
                           d=1, D=1, trace=True,
                           error_action='ignore',  
                           suppress_warnings=True, 
                           stepwise=True)

Fit ARIMA: order=(1, 1, 1) seasonal_order=(0, 1, 1, 12); AIC=15225.073, BIC=15252.543, Fit time=23.129 seconds
Fit ARIMA: order=(0, 1, 0) seasonal_order=(0, 1, 0, 12); AIC=16562.196, BIC=16573.184, Fit time=0.163 seconds
Fit ARIMA: order=(1, 1, 0) seasonal_order=(1, 1, 0, 12); AIC=15950.127, BIC=15972.103, Fit time=5.445 seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(0, 1, 1, 12); AIC=15223.546, BIC=15245.521, Fit time=16.640 seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(1, 1, 1, 12); AIC=15224.418, BIC=15251.887, Fit time=21.261 seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(0, 1, 0, 12); AIC=16462.007, BIC=16478.489, Fit time=1.535 seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(0, 1, 2, 12); AIC=15224.373, BIC=15251.843, Fit time=41.026 seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(1, 1, 2, 12); AIC=15227.156, BIC=15260.119, Fit time=35.887 seconds
Fit ARIMA: order=(0, 1, 0) seasonal_order=(0, 1, 1, 12); AIC=15310.107, BIC=15326.589, Fit time=10.599 seconds
Fit 

In [49]:
stepwise_model.aic()

15223.545926823597

In [50]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1810 entries, 2015-08-07 to 2020-07-20
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   High    1810 non-null   float64
dtypes: float64(1)
memory usage: 28.3 KB


In [65]:
train = data.loc['2015-08-07':'2020-07-9']
train.tail()

,High
Date,
2020-07-05,229.86
2020-07-06,242.13
2020-07-07,242.68
2020-07-08,248.31
2020-07-09,247.29


In [66]:
test = data.loc['2020-07-10':]
test.head()

,High
Date,
2020-07-10,242.88
2020-07-11,241.53
2020-07-12,243.31
2020-07-13,244.31
2020-07-14,242.00


In [67]:
len(test)

11

In [69]:
stepwise_model.fit(train)

ARIMA(callback=None, disp=0, maxiter=50, method=None, order=(0, 1, 1),
      out_of_sample_size=0, scoring='mse', scoring_args={},
      seasonal_order=(0, 1, 1, 12), solver='lbfgs', start_params=None,
      suppress_warnings=True, transparams=True, trend='c')

In [70]:
future_forecast = stepwise_model.predict(n_periods=14)

In [71]:
future_forecast

array([246.50860562, 244.9137549 , 244.25325862, 245.27775365,
       247.48942691, 246.92227009, 247.38924032, 246.25967137,
       246.89436181, 246.17194753, 246.27936301, 245.20598982,
       244.70985575, 243.1112394 ])

In [72]:
test['Prediction'] = future_forecast[0:11].tolist()
test

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,High,Prediction
Date,,
2020-07-10,242.88,246.508606
2020-07-11,241.53,244.913755
2020-07-12,243.31,244.253259
2020-07-13,244.31,245.277754
2020-07-14,242.00,247.489427
2020-07-15,241.40,246.922270
2020-07-16,239.01,247.389240
2020-07-17,234.42,246.259671
2020-07-18,236.54,246.894362
